## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [134]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [135]:
# x_train = (5000, 32, 32, 3) y_train = clifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0, 1, 2, 3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [136]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

/Users/wasn/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [137]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
# 預設 Ｖalid (多得刪除)
# (3*3*3+1)*32 = 896

classifier.add(BatchNormalization())
# out_height = ceil(float(in_height)/float(strides[1])) = cile(30/1) = 30
# out_width = ceil(float(in_width)/float(strides[1])) = cile(30/1) = 30
# 32*4 = 128 (trainable parameters=64)

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same'))
# (3*3*32+1)*32 = 9248
classifier.add(BatchNormalization())

classifier.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_60 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 15, 15, 32)        9248      
_________________________________________________________________
batch_normalization_61 (Batc (None, 15, 15, 32)        128       
Total params: 10,400
Trainable params: 10,272
Non-trainable params: 128
_________________________________________________________________


In [138]:
#卷積組合
classifier.add(Convolution2D(32, kernel_size=(5, 5), padding='same'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

classifier.summary()



Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_60 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 15, 15, 32)        9248      
_________________________________________________________________
batch_normalization_61 (Batc (None, 15, 15, 32)        128       
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 15, 15, 32)        25632     
_________________________________________________________________
batch_normalization_62 (Batc (None, 15, 15, 32)      

/Users/wasn/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  del sys.path[0]


In [139]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 112s 2ms/step - loss: 1.3506 - accuracy: 0.5408
Epoch 2/100
50000/50000 [==============================] - 123s 2ms/step - loss: 0.9702 - accuracy: 0.6603
Epoch 3/100
50000/50000 [==============================] - 122s 2ms/step - loss: 0.8116 - accuracy: 0.7159
Epoch 4/100
50000/50000 [==============================] - 2281s 46ms/step - loss: 0.7105 - accuracy: 0.7505
Epoch 5/100
50000/50000 [==============================] - 109s 2ms/step - loss: 0.6208 - accuracy: 0.7833
Epoch 6/100
50000/50000 [==============================] - 165s 3ms/step - loss: 0.5402 - accuracy: 0.8100
Epoch 7/100
50000/50000 [==============================] - 346s 7ms/step - loss: 0.4549 - accuracy: 0.8411
Epoch 8/100
50000/50000 [==============================] - 122s 2ms/step - loss: 0.3857 - accuracy: 0.8657
Epoch 9/100
50000/50000 [==============================] - 128s 3ms/step - loss: 0.3240 - accuracy: 0.8869
Epoch 10/100
50000/50000 [=========

50000/50000 [==============================] - 92s 2ms/step - loss: 0.0203 - accuracy: 0.9935
Epoch 78/100
50000/50000 [==============================] - 92s 2ms/step - loss: 0.0236 - accuracy: 0.9927
Epoch 79/100
50000/50000 [==============================] - 91s 2ms/step - loss: 0.0295 - accuracy: 0.9908
Epoch 80/100
50000/50000 [==============================] - 92s 2ms/step - loss: 0.0226 - accuracy: 0.9928
Epoch 81/100
50000/50000 [==============================] - 92s 2ms/step - loss: 0.0243 - accuracy: 0.9928
Epoch 82/100
50000/50000 [==============================] - 89s 2ms/step - loss: 0.0160 - accuracy: 0.9946
Epoch 83/100
50000/50000 [==============================] - 89s 2ms/step - loss: 0.0263 - accuracy: 0.9918
Epoch 84/100
50000/50000 [==============================] - 89s 2ms/step - loss: 0.0228 - accuracy: 0.9926
Epoch 85/100
50000/50000 [==============================] - 89s 2ms/step - loss: 0.0246 - accuracy: 0.9915
Epoch 86/100
50000/50000 [========================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [140]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.0379672e-07, 6.9383253e-12, 1.8746190e-05, 9.9952090e-01,
        4.6001925e-04, 2.6850355e-10, 2.4943953e-12, 2.2559048e-10,
        8.8439645e-14, 1.4995011e-10]], dtype=float32)